# Concat issues

MyHDL handles booleans as concat arg, but does not resolve to valid VHDL.

In [1]:
import myhdl

Signal = myhdl.Signal
modbv = myhdl.modbv

@myhdl.block
def unit(clk):
    a = [ Signal(bool() ) for _ in range(4) ]
    b = Signal(modbv()[4:])
    
    c = Signal(modbv(0)[4:])

    @myhdl.always_comb
    def worker():
        b.next = myhdl.concat(c >= 0x8, c[3:])
        
    @myhdl.always(clk.posedge)
    def counter():
        c.next = c + 1
        
    return worker, counter

clk = Signal(bool())
inst = unit(clk)
inst.convert("VHDL")

/home/cyrite/.local/lib/python3.10/site-packages/myhdl/conversion/_toVHDL.py:471: ToVHDLWarning: Signal is driven but not read: b
  warnings.warn("%s: %s" % (_error.UnreadSignal, s._name),


In [2]:
@myhdl.block
def tb():
    a, b = [ Signal(bool()) for _ in range(2) ]
    c, d = [ Signal(bool(), delay = 4) for _ in range(2) ]
    @myhdl.instance
    def main():
        a.next = False
        yield myhdl.delay(2)
        a.next = True
        
    @myhdl.always_comb
    def assign():
        c.next = a

    return myhdl.instances()

inst = tb()
inst.convert("VHDL")    
    

/home/cyrite/.local/lib/python3.10/site-packages/myhdl/conversion/_toVHDL.py:471: ToVHDLWarning: Signal is driven but not read: c
  warnings.warn("%s: %s" % (_error.UnreadSignal, s._name),


In [3]:
!cat -n tb.vhd

     1	-- File: tb.vhd
     2	-- Generated by MyHDL 0.11.49
     3	-- Date: Wed Jul 17 20:08:54 2024
     4	
     5	
     6	library IEEE;
     7	use IEEE.std_logic_1164.all;
     8	use IEEE.numeric_std.all;
     9	use std.textio.all;
    10	
    11	use work.pck_myhdl_011.all;
    12	
    13	entity tb is
    14	end entity tb;
    15	
    16	
    17	architecture MyHDL of tb is
    18	
    19	
    20	
    21	signal a: std_logic;
    22	signal c: std_logic;
    23	
    24	begin
    25	
    26	
    27	
    28	
    29	TB_MAIN: process is
    30	begin
    31	    a <= '0';
    32	    wait for 2 * 1 ns;
    33	    a <= '1';
    34	    wait;
    35	end process TB_MAIN;
    36	
    37	
    38	c <= a;
    39	
    40	end architecture MyHDL;


In [4]:
! ghdl -i unit.vhd pck_myhdl_011.vhd

In [5]:
! ghdl -m unit

unit.vhd:33:25:error: no function declarations for operator "&"


# MyIRL emulation

In [6]:
from myirl.emulation.myhdl import *
from myirl.test.common_test import run_ghdl

from myirl.wire import SLV

@block
def unit1(clk):
    a = [ Signal(bool() ) for _ in range(4) ]
    b = Signal(modbv()[4:])
    
    c = Signal(modbv(0)[4:])

    
    @always_comb
    def worker():
        b.next = concat(c >= 0x8, c == 0, c[2:])
        
    @always(clk.posedge)
    def counter():
        c.next = c + 1
        
    return instances()

In [7]:
def convert():
    clk = Signal(bool())
    inst = unit1(clk)
    files = inst.elab(targets.VHDL)
    run_ghdl(files, inst, std="93c", debug = True)
    return files
f = convert()

/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/components.py:215: UserWarning: @component `unit1` interface :Unspecified port I/O 'clk' => IN
  base.warnings.warn((msg + " => IN") % n)


 Writing 'unit1' to file /tmp/myirl_unit1_rznah5c6/unit1.vhdl 
WORK DIR of instance [Instance unit1 I/F: [// ID: unit1_0 ]] /tmp/myirl_unit1_rznah5c6/
==== COSIM stdout ====
../../src/ieee/v93/numeric_std-body.vhdl:1461:7:@0ms:(assertion warning): NUMERIC_STD.">=": metavalue detected, returning FALSE
../../src/ieee/v93/numeric_std-body.vhdl:1613:7:@0ms:(assertion warning): NUMERIC_STD."=": metavalue detected, returning FALSE



In [8]:
! cat {f[0]}

-- File generated from source:
--     /tmp/ipykernel_96221/2136329268.py
-- (c) 2016-2022 section5.ch
-- Modifications may be lost, edit the source file instead.

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity unit1 is
    port (
        clk : in std_ulogic
    );
end entity unit1;

architecture cyriteHDL of unit1 is
    -- Local type declarations
    -- Signal declarations
    signal b : unsigned(3 downto 0);
    signal c : unsigned(3 downto 0);
begin
    
worker:
    process(c)
    begin
        b <= (std_ulogic'(from_bool((c >= x"8"))) & std_ulogic'(from_bool((c = x"0"))) & c(2-1 downto 0));
    end process;

    
counter:
    process(clk)
    begin
        if rising_edge(clk) then
            c <= resize((resize(c, 5) + 1), 4);
        end if;
    end process;
end architecture cyriteHDL;

